In [1]:
import pandas as pd
import numpy as np
import re
from random import random
from tqdm import notebook
def tqdm(x, **kargs):
    return notebook.tqdm(x, leave=False, **kargs)
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import torch
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
from simpletransformers.classification import MultiLabelClassificationModel, ClassificationModel
from sklearn.preprocessing import MultiLabelBinarizer

I0704 04:49:53.962123 140307551852352 file_utils.py:39] PyTorch version 1.5.0 available.


In [3]:
from simpletransformers.language_modeling import LanguageModelingModel

## Read input data

In [4]:
# df = pd.read_json('./source/train_gold.json', lines=True)
df = pd.read_json('./preprocessed/preprocess_train.json', lines=True)
df_new = df.copy()
df_new.head()

,idx,text,reply,categories
0,0,we can all agree that any song by Niall Horan .,oui oui,[yes]
1,1,Will you be install # Scotty from marketing ...,,[no]
2,2,Growing up my mum would call me a Nigga despit...,And he joins in ? ? ? Pour some hot grits o...,[smh]
3,3,Rest your head on my chest when the world feel...,face with tear of joy,[wink]
4,4,Imagine Will Hernandez and Wills both doing a ...,,[yes]


In [5]:
categories_type = pd.read_json('./source/categories.json', lines=True)
categories_mapping = {v[0]: k for k, v in categories_type.to_dict('list').items()}

In [6]:
df_dev_ori = pd.read_json('./source/dev_unlabeled.json', lines=True)
df_dev = pd.read_json('./preprocessed/preprocess_dev.json', lines=True)
df_dev_result = df_dev.copy()[['text', 'reply']]
df_dev_result.head()

,text,reply
0,"Drop your cash app , use hashtag # BailoutHu...",$ tyratomaro # BailoutHumans
1,After interviewing with a few incredible peopl...,CONGRATS ! ! ! ! !
2,I know GTC festival not happening next month b...,
3,"Lordy , my daughter just said , "" I wonde...",
4,THE UNEMPLOYMENT CLAIM SYSTEM SUCKS SO MUCH DICK,Watching everyone else get their weekly unempl...


In [7]:
df_test_ori = pd.read_json('./source/test_unlabeled.json', lines=True)
df_test = pd.read_json('./preprocessed/preprocess_test.json', lines=True)
df_test_result = df_test.copy()[['text', 'reply']]
df_test_result.head()

,text,reply
0,@ Youngdeji - I think if uzi and carti dro...,
1,For the third year in a row we are discussing ...,
2,dababy album sounds like it was made for nigga...,That ' s why you buy it .
3,Majority of Indians do not watch any sport oth...,@ ZairaWasimmm got a great story because of t...
4,everybody is just now listen to @ madisonbeer...,


### Use multi-hot encoding and change column name

In [8]:
mlb = MultiLabelBinarizer()
multi_hot = mlb.fit_transform(df_new['categories'].values)
multi_hot_list = [list(_) for _ in multi_hot]

In [9]:
df_new['labels'] = multi_hot_list
df_new = df_new[['text', 'reply', 'labels']]
df_new.columns = ['text_a', 'text_b', 'labels']
df_dev_result.columns = ['text_a', 'text_b']
df_test_result.columns = ['text_a', 'text_b']

## Train 10 roberta language models and fine-tune to multi-label classification model

In [12]:
# Output data to train_file for training LM with RoBERTa format
df_forLM = df_new.copy()
df_forLM['text'] = ['<s> '] + df_new['text_a'] + [' </s></s> '] + df_new['text_b'] + [' <s>']
df_forLM['text'].to_csv(r'LM/training.txt', header=None, index=None, sep=' ')

for i in range(1, 11):
    # Enhanced pre-trained language model
    seq = 113
    block_size = 96
    train_lm_args = {
        "output_dir": "round2_blend_model/round2_LM/outputs_roberta_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "LM/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "block_size": block_size,
        "max_seq_length": seq,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "save_model_every_epoch": False,
    }
    # Fine-tuned to multi-label classification model
    MLR_seq = 113
    model_lm = LanguageModelingModel("roberta", "roberta-base", args=train_lm_args)
    model_lm.train_model('LM/training.txt')
    train_args = {
        "output_dir": "round2_blend_model/round2_MLR/outputs_roberta_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "model_results_gpu/outputs/best_model/",
        
        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": MLR_seq,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "save_model_every_epoch": False,

        "overwrite_output_dir": False,
        "reprocess_input_data": True,

        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,

        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,

        "manual_seed": None,
        "encoding": None,
    }
    model = MultiLabelClassificationModel('roberta', 'round2_blend_model/round2_LM/outputs_roberta_LM_training{}/'.format(i), num_labels=43, args=train_args)
    model.train_model(df_new)
    result, model_outputs, wrong_predictions = model.eval_model(df_new)

I0615 21:17:36.069663 140303622817600 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at cache/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
I0615 21:17:36.070304 140303622817600 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at cache/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
I0615 21:17:37.189713 140303622817600 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at cache/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
I0615 21:17:37.190888 140303622817600 configuration_utils.py:321] Mode

I0615 21:17:56.828726 140303622817600 language_modeling_utils.py:229]  Saving features into cached file cache/roberta_cached_lm_93_training.txt
I0615 21:17:56.891100 140303622817600 language_modeling_model.py:473]  Training started


Running loss: 2.759356


Running loss: 1.996590


Running loss: 2.135384

...

Running loss: 0.138657

I0616 00:34:20.887150 140303622817600 configuration_utils.py:144] Configuration saved in round2_blend_model/round2_MLR/outputs_roberta_LM_training15/checkpoint-8000/config.json


Running loss: 0.081928

I0616 00:34:21.294524 140303622817600 modeling_utils.py:483] Model weights saved in round2_blend_model/round2_MLR/outputs_roberta_LM_training15/checkpoint-8000/pytorch_model.bin
I0616 00:34:22.320711 140303622817600 configuration_utils.py:144] Configuration saved in round2_blend_model/round2_MLR/outputs_roberta_LM_training15/config.json


I0616 00:34:22.663582 140303622817600 modeling_utils.py:483] Model weights saved in round2_blend_model/round2_MLR/outputs_roberta_LM_training15/pytorch_model.bin
I0616 00:34:22.728643 140303622817600 classification_model.py:279]  Training of roberta model complete. Saved to round2_blend_model/round2_MLR/outputs_roberta_LM_training15/.


## Train 5 BERT-base-cased language models and fine-tune to multi-label classification model

In [12]:
# Output data to train_file for training LM with BERT format
df_forLM = df_new.copy()
df_forLM['text'] = ['[CLS] '] + df_new['text_a'] + [' [SEP] '] + df_new['text_b'] + [' [SEP]']
df_forLM['text'].to_csv(r'LM/training.txt', header=None, index=None, sep=' ')
for i in range(1, 6):
    # Enhanced pre-trained language model
    seq = 113
    block_size = 96
    train_lm_args = {
        "output_dir": "round2_blend_model/round2_LM/outputs_bert_cased_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "LM/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "block_size": block_size,
        "max_seq_length": seq,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "save_model_every_epoch": False,
    }
    # Fine-tuned to multi-label classification model
    MLR_seq = 113
    model_lm = LanguageModelingModel("bert", "bert-base-cased", args=train_lm_args)
    model_lm.train_model('LM/training.txt')
    train_args = {
        "output_dir": "round2_blend_model/round2_MLR/outputs_bert_cased_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "model_results_gpu/outputs/best_model/",
        
        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": MLR_seq,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "save_model_every_epoch": False,

        "overwrite_output_dir": False,
        "reprocess_input_data": True,

        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,

        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,

        "manual_seed": None,
        "encoding": None,
    }
    model = MultiLabelClassificationModel('bert', 'round2_blend_model/round2_LM/outputs_bert_cased_LM_training{}/'.format(i), num_labels=43, args=train_args)
    model.train_model(df_new)
    result, model_outputs, wrong_predictions = model.eval_model(df_new)

I0616 07:05:07.455054 139967496189760 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0616 07:05:08.484836 139967496189760 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at cache/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0616 07:05:08.485928 139967496189760 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings

I0616 07:05:22.044003 139967496189760 language_modeling_utils.py:229]  Saving features into cached file cache/bert_cached_lm_94_training.txt
I0616 07:05:22.109065 139967496189760 language_modeling_model.py:473]  Training started


Running loss: 2.350355


Running loss: 1.203791


Running loss: 2.207254

...

Running loss: 0.095023

I0616 09:59:51.396079 139967496189760 modeling_utils.py:483] Model weights saved in round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training10/checkpoint-6000/pytorch_model.bin


Running loss: 0.090323

I0616 10:05:31.843841 139967496189760 configuration_utils.py:144] Configuration saved in round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training10/checkpoint-8000/config.json
I0616 10:05:32.235633 139967496189760 modeling_utils.py:483] Model weights saved in round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training10/checkpoint-8000/pytorch_model.bin
I0616 10:05:32.901341 139967496189760 configuration_utils.py:144] Configuration saved in round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training10/config.json


I0616 10:05:33.282076 139967496189760 modeling_utils.py:483] Model weights saved in round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training10/pytorch_model.bin
I0616 10:05:33.308022 139967496189760 classification_model.py:279]  Training of bert model complete. Saved to round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training10/.


## Train 5 BERT-base-uncased language models and fine-tune to multi-label classification model

In [ ]:
# Output data to train_file for training LM with BERT format
df_forLM = df_new.copy()
df_forLM['text'] = ['[CLS] '] + df_new['text_a'] + [' [SEP] '] + df_new['text_b'] + [' [SEP]']
df_forLM['text'].to_csv(r'LM/training.txt', header=None, index=None, sep=' ')
for i in range(1, 6):
    # Enhanced pre-trained language model
    seq = 113
    block_size = 96
    train_lm_args = {
        "output_dir": "round2_blend_model/round2_LM/outputs_bert_uncased_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "LM/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "block_size": block_size,
        "max_seq_length": seq,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "save_model_every_epoch": False,
    }
    # Fine-tuned to multi-label classification model
    MLR_seq = 113
    model_lm = LanguageModelingModel("bert", "bert-base-uncased", args=train_lm_args)
    model_lm.train_model('LM/training.txt')
    train_args = {
        "output_dir": "round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "model_results_gpu/outputs/best_model/",
        
        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": MLR_seq,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "save_model_every_epoch": False,

        "overwrite_output_dir": False,
        "reprocess_input_data": True,

        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,

        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,

        "manual_seed": None,
        "encoding": None,
    }
    model = MultiLabelClassificationModel('bert', 'round2_blend_model/round2_LM/outputs_bert_uncased_LM_training{}/'.format(i), num_labels=43, args=train_args)
    model.train_model(df_new)
    result, model_outputs, wrong_predictions = model.eval_model(df_new)

## Power n weighted sum to blend models, average same model type first

In [10]:
import csv
from os import listdir
from os.path import isfile
from operator import add 

### Predicting dev data

In [15]:
to_predict = []
for i in range(len(df_dev_result)):
    # for roberta format
    text_reply = '<s> ' + df_dev_result['text_a'][i] + ' </s></s> ' + df_dev_result['text_b'][i] + ' <s>'
    to_predict.append(text_reply)

In [16]:
for model_idx in range(1, 16):
    model_roberta_base = MultiLabelClassificationModel('roberta', 'round2_blend_model/round2_MLR/outputs_roberta_LM_training' + str(model_idx) + '/')
    predictions_roberta_base, raw_outputs_roberta_base = model_roberta_base.predict(to_predict)
    with open("./round2_blend_model/round2_results/dev/roberta_" + str(model_idx) + ".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(raw_outputs_roberta_base)

I0616 10:11:23.481842 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training1/config.json
I0616 10:11:23.483280 139967496189760 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0616 10:11:47.132991 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training2/config.json
I0616 10:11:47.134205 139967496189760 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0616 10:12:09.550442 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training3/config.json
I0616 10:12:09.552167 139967496189760 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0616 10:12:30.891720 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training4/config.json
I0616 10:12:30.892633 139967496189760 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0616 10:12:52.374381 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training5/config.json
I0616 10:12:52.376261 139967496189760 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

...

I0616 10:14:58.845094 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training11/config.json
I0616 10:14:58.846000 139967496189760 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "2

I0616 10:15:20.807399 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training12/config.json
I0616 10:15:20.808599 139967496189760 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "2

I0616 10:15:42.016558 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training13/config.json
I0616 10:15:42.017987 139967496189760 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "2

I0616 10:16:02.905007 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training14/config.json
I0616 10:16:02.907650 139967496189760 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "2

I0616 10:16:24.304792 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training15/config.json
I0616 10:16:24.306084 139967496189760 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "2

In [17]:
to_predict = []
for i in range(len(df_dev_result)):
#     for bert format
    text_reply = '[CLS] ' + df_dev_result['text_a'][i] + ' [SEP] ' + df_dev_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [18]:
for bert_type in ['uncased', 'cased']:
    for model_idx in range(1, 11):
        model_bert_base = MultiLabelClassificationModel('bert', 'round2_blend_model/round2_MLR/outputs_bert_' + bert_type + '_LM_training' + str(model_idx) + '/')
        predictions_bert_base, raw_outputs_bert_base = model_bert_base.predict(to_predict)
        with open("./round2_blend_model/round2_results/dev/bert_" + bert_type + "_" + str(model_idx) + ".csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(raw_outputs_bert_base)

I0616 10:19:26.670245 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training1/config.json
I0616 10:19:26.671617 139967496189760 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0616 10:19:46.744601 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training2/config.json
I0616 10:19:46.745575 139967496189760 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0616 10:20:07.330178 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training3/config.json
I0616 10:20:07.331279 139967496189760 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0616 10:20:27.917029 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training4/config.json
I0616 10:20:27.919538 139967496189760 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0616 10:20:48.785302 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training5/config.json
I0616 10:20:48.786776 139967496189760 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

...

I0616 10:24:26.721914 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training6/config.json
I0616 10:24:26.723083 139967496189760 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0616 10:24:47.584363 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training7/config.json
I0616 10:24:47.585210 139967496189760 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0616 10:25:08.242923 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training8/config.json
I0616 10:25:08.244040 139967496189760 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0616 10:25:29.519124 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training9/config.json
I0616 10:25:29.520027 139967496189760 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0616 10:25:50.892597 139967496189760 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training10/config.json
I0616 10:25:50.898528 139967496189760 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25

In [64]:
files = listdir('./round2_blend_model/round2_results/dev/')
roberta_prob = np.zeros([len(df_dev_result), len(categories_mapping)])
bert_uncase_prob = np.zeros([len(df_dev_result), len(categories_mapping)])
bert_case_prob = np.zeros([len(df_dev_result), len(categories_mapping)])
for f in files:
    if f.find('.') != 0:
        model_prob = pd.read_csv('./round2_blend_model/round2_results/dev/' + f, header=None)
        probability_list = model_prob.values.tolist()
        if f.find('roberta') >= 0:
            roberta_prob = np.add(roberta_prob, probability_list)
        if f.find('_cased_') >= 0:
            bert_case_prob = np.add(bert_case_prob, probability_list)
        if f.find('uncased') >= 0:
            bert_uncase_prob = np.add(bert_uncase_prob, probability_list)

In [65]:
roberta_count = 15
bert_count = 10
roberta_prob /= roberta_count
bert_case_prob /= bert_count
bert_uncase_prob /= bert_count

In [66]:
P = 1.8
total_predict = []
for eid, row_roberta_base in enumerate(roberta_prob):
    row = (row_roberta_base ** P) * 3.0 + \
            (bert_case_prob[eid] ** P) * 1.8 + (bert_uncase_prob[eid] ** P) * 0.8 
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(row)), key=lambda k: row[k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            predict_class.append(key)
            
    total_predict.append(predict_class)

In [67]:
df_dev_ori['categories'] = total_predict
df_dev_ori.head()

,idx,text,reply,categories
0,32000,"Drop your cash app, use hashtag #BailoutHumansNow",$tyratomaro #BailoutHumans,"[applause, dance, good_luck, happy_dance, plea..."
1,32001,After interviewing with a few incredible peopl...,CONGRATS!!!!!,"[agree, applause, happy_dance, slow_clap, thum..."
2,32002,I know GTC festival not happening next month b...,,"[agree, facepalm, idk, no, sigh, yes]"
3,32003,"Lordy, my daughter just said, “I wonder how th...",,"[facepalm, omg, seriously, shocked, sigh, smh]"
4,32004,THE UNEMPLOYMENT CLAIM SYSTEM SUCKS SO MUCH DICK,Watching everyone else get their weekly unempl...,"[agree, applause, no, popcorn, seriously, yes]"


In [68]:
df_dev_ori.to_json('./round2_results/dev.json', orient='records', lines=True)

### Predicting test data

In [50]:
to_predict = []
for i in range(len(df_test_result)):
    # for roberta format
    text_reply = '<s> ' + df_test_result['text_a'][i] + ' </s></s> ' + df_test_result['text_b'][i] + ' <s>'
    to_predict.append(text_reply)

In [51]:
for model_idx in range(1, 11):
    model_roberta_base = MultiLabelClassificationModel('roberta', 'round2_blend_model/round2_MLR/outputs_roberta_LM_training' + str(model_idx) + '/')
    predictions_roberta_base, raw_outputs_roberta_base = model_roberta_base.predict(to_predict)
    with open("./round2_blend_model/round2_results/test/roberta_" + str(model_idx) + ".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(raw_outputs_roberta_base)

I0615 11:07:37.106949 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training1/config.json
I0615 11:07:37.107669 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:07:53.573905 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training2/config.json
I0615 11:07:53.574656 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:08:09.962673 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training3/config.json
I0615 11:08:09.963291 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:08:12.493335 140378694625088 tokenization_utils.py:929] Model name 'round2_blend_model/round2_MLR/outputs_roberta_LM_training3/' not found in model shortcut name list (roberta-base, roberta-large, roberta-large-mnli, distilroberta-base, roberta-base-openai-detector, roberta-large-openai-detector). Assuming 'round2_blend_model/round2_MLR/outputs_roberta_LM_training3/' is a path, a model identifier, or url to a directory containing tokenizer files.
I0615 11:08:12.493974 140378694625088 tokenization_utils.py:958] Didn't find file round2_blend_model/round2_MLR/outputs_roberta_LM_training3/added_tokens.json. We won't load it.
I0615 11:08:12.494538 140378694625088 tokenization_utils.py:1013] loading file round2_blend_model/round2_MLR/outputs_roberta_LM_training3/vocab.json
I0615 11:08:12.494816 140378694625088 tokenization_utils.py:1013] loading file round2_blend_model/round2_MLR/outputs_roberta_LM_training3/merges.txt
I0615 11:08:12.495084 140378694625088 tokenization_utils.py:1013

I0615 11:08:26.358447 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training4/config.json
I0615 11:08:26.359086 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

...

I0615 11:08:59.294439 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training6/config.json
I0615 11:08:59.295136 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:09:15.665878 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training7/config.json
I0615 11:09:15.666563 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:09:32.071654 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training8/config.json
I0615 11:09:32.072266 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:09:48.618046 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training9/config.json
I0615 11:09:48.618675 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:10:05.035332 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training10/config.json
I0615 11:10:05.035945 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "2

In [52]:
to_predict = []
for i in range(len(df_test_result)):
#     for bert format
    text_reply = '[CLS] ' + df_test_result['text_a'][i] + ' [SEP] ' + df_test_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [53]:
for bert_type in ['uncased', 'cased']:
    for model_idx in range(1, 6):
        model_bert_base = MultiLabelClassificationModel('bert', 'round2_blend_model/round2_MLR/outputs_bert_' + bert_type + '_LM_training' + str(model_idx) + '/')
        predictions_bert_base, raw_outputs_bert_base = model_bert_base.predict(to_predict)
        with open("./round2_blend_model/round2_results/test/bert_" + bert_type + "_" + str(model_idx) + ".csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(raw_outputs_bert_base)

I0615 11:14:29.435397 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training1/config.json
I0615 11:14:29.436059 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:14:45.214638 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training2/config.json
I0615 11:14:45.215296 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:15:01.142184 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training3/config.json
I0615 11:15:01.142876 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:15:16.941130 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training4/config.json
I0615 11:15:16.941755 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:15:18.688717 140378694625088 tokenization_utils.py:929] Model name 'round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training4/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming 'round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training4/' is a path, a model identifier, or url to a directory containing tokenizer files.
I0615 11:15:18.689321 140378694625088 tokenization_utils.py:958] Didn't find file round2_blend_model/round2_MLR/outputs_be

...

I0615 11:16:04.541762 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training2/config.json
I0615 11:16:04.542450 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0615 11:16:20.400134 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training3/config.json
I0615 11:16:20.400911 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0615 11:16:36.364712 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training4/config.json


I0615 11:16:36.365415 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": "LABEL_28",
    "29": "LABEL_29",
    "30": "LABEL_30",
    "31": "LABEL_31",
    "32": "LABEL_32",

I0615 11:16:52.200727 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training5/config.json
I0615 11:16:52.201379 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

In [11]:
files = listdir('./round2_blend_model/round2_results/test/')
roberta_prob = np.zeros([len(df_test_result), len(categories_mapping)])
bert_uncase_prob = np.zeros([len(df_test_result), len(categories_mapping)])
bert_case_prob = np.zeros([len(df_test_result), len(categories_mapping)])
for f in files:
    if f.find('.') != 0:
        model_prob = pd.read_csv('./round2_blend_model/round2_results/test/' + f, header=None)
        probability_list = model_prob.values.tolist()
        if f.find('roberta') >= 0:
            roberta_prob = np.add(roberta_prob, probability_list)
        if f.find('_cased_') >= 0:
            bert_case_prob = np.add(bert_case_prob, probability_list)
        if f.find('uncased') >= 0:
            bert_uncase_prob = np.add(bert_uncase_prob, probability_list)

In [12]:
roberta_count = 10
bert_count = 5
roberta_prob /= roberta_count
bert_case_prob /= bert_count
bert_uncase_prob /= bert_count

In [13]:
P = 1.8
total_predict = []
for eid, row_roberta_base in enumerate(roberta_prob):
    row = (row_roberta_base ** P) * 3.0 + \
            (bert_case_prob[eid] ** P) * 1.8 + (bert_uncase_prob[eid] ** P) * 0.8 
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(row)), key=lambda k: row[k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            predict_class.append(key)
            
    total_predict.append(predict_class)

In [14]:
df_test_ori['categories'] = total_predict
df_test_ori.head()

,idx,text,reply,categories
0,36000,@Youngdeji_ I think if uzi and carti dropping ...,,"[agree, dance, happy_dance, no, please, yes]"
1,36001,For the third year in a row we’re discussing t...,,"[facepalm, no, seriously, sigh, smh, yes]"
2,36002,dababy album sounds like it was made for nigga...,That's why you bought it.,"[agree, idk, no, seriously, smh, yes]"
3,36003,Majority of Indians don’t watch any sport othe...,@ZairaWasimmm got a great story because of the...,"[agree, applause, awww, shocked, slow_clap, th..."
4,36004,everybody is just now listening to @madisonbee...,,"[agree, applause, facepalm, idk, seriously, smh]"


In [15]:
df_test_ori.to_json('./round2_results/eval.json', orient='records', lines=True)

## Below are statistics

In [8]:
from transformers import RobertaTokenizer

### Count min, max, and average text length to select proper max_seq_length

In [9]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
text_cnt = 0
max_cnt = 0
min_cnt = 1e3
text_bar = []
for each_text in df['text']:
    each_cnt = len(tokenizer.tokenize(each_text))
    text_cnt += each_cnt
    text_bar.append(each_cnt // 10)
    if each_cnt > max_cnt:
        max_cnt = each_cnt
    if each_cnt < min_cnt:
        min_cnt = each_cnt
text_level = Counter(sorted(text_bar))
print("Max text length is: {}".format(max_cnt))
print("Min text length is: {}".format(min_cnt))
print("Average text length is: {}".format(text_cnt / len(df['text'])))

I0704 04:50:51.059964 140307551852352 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /home/ino/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
I0704 04:50:51.061814 140307551852352 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at /home/ino/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda


Max text length is: 507
Min text length is: 1
Average text length is: 34.4776875


In [10]:
reply_cnt = 0
max_cnt = 0
min_cnt = 1e3
reply_bar = []
for each_reply in df['reply']:
    each_cnt = len(tokenizer.tokenize(each_reply))
    reply_cnt += each_cnt
    reply_bar.append(each_cnt // 10)
    if each_cnt > max_cnt:
        max_cnt = each_cnt
    if each_cnt < min_cnt:
        min_cnt = each_cnt
reply_level = Counter(sorted(reply_bar))
print("Max reply length is: {}".format(max_cnt))
print("Min reply length is: {}".format(min_cnt))
print("Average reply length is: {}".format(reply_cnt / len(df['reply'])))

Max reply length is: 242
Min reply length is: 0
Average reply length is: 6.30953125


In [11]:
text_reply_cnt = 0
max_cnt = 0
min_cnt = 1e3
text_reply_bar = []
for eid, each_reply in enumerate(df['reply']):
    each_text_cnt = len(tokenizer.tokenize(df['text'][eid]))
    each_reply_cnt = len(tokenizer.tokenize(each_reply))
    each_cnt = each_text_cnt + each_reply_cnt
    text_reply_bar.append(each_cnt // 10)
    text_reply_cnt += each_cnt
    if each_cnt > max_cnt:
        max_cnt = each_cnt
    if each_cnt < min_cnt:
        min_cnt = each_cnt
text_reply_level = Counter(sorted(text_reply_bar))
print("Max text + reply length is: {}".format(max_cnt))
print("Min text + reply length is: {}".format(min_cnt))
print("Average text + reply length is: {}".format(text_reply_cnt / len(df['reply'])))

Max text + reply length is: 507
Min text + reply length is: 1
Average text + reply length is: 40.78721875
